## **1) Importing Python Packages for GAN**


In [1]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import BatchNormalization, Dense, Reshape, Flatten, LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
import os

# Membuat folder jika belum ada
os.makedirs("generated_images", exist_ok=True)


Pertama-tama, saya mengimpor seluruh library yang dibutuhkan untuk membangun dan melatih model Generative Adversarial Network (GAN). Library yang digunakan meliputi mnist dari Keras untuk memuat dataset tulisan tangan digit, kemudian Sequential, Dense, BatchNormalization, Reshape, Flatten, dan LeakyReLU yang digunakan untuk membentuk arsitektur jaringan saraf. Saya juga menggunakan optimizer Adam dari TensorFlow untuk mempercepat proses pembelajaran model, serta numpy untuk melakukan operasi numerik. Selain itu, saya memanfaatkan library os untuk membuat folder bernama generated_images, yang berfungsi sebagai tempat penyimpanan gambar hasil keluaran dari generator. Pembuatan folder dilakukan secara otomatis dengan perintah os.makedirs("generated_images", exist_ok=True) agar folder dibuat hanya jika belum ada sebelumnya.

## **2) Variables for Neural Networks & Data**

In [2]:
img_width = 28
img_height = 28
channels = 1
img_shape = (img_width, img_height, channels)
latent_dim = 100

# gunakan parameter baru
adam = Adam(learning_rate=0.0001)


Selanjutnya, saya mendefinisikan parameter dasar yang digunakan dalam konfigurasi model. Ukuran gambar ditetapkan sebesar 28x28 piksel dengan satu kanal warna (grayscale), karena dataset MNIST memang memiliki format tersebut. Variabel latent_dim ditetapkan bernilai 100, yang berarti generator akan menerima input berupa vektor acak berdimensi 100 untuk menghasilkan gambar baru. Kemudian, saya mendefinisikan optimizer Adam dengan learning rate sebesar 0.0001, karena nilai ini relatif kecil dan membantu proses pelatihan menjadi lebih stabil serta menghindari perubahan bobot yang terlalu drastis pada setiap iterasi.

## **3) Building Generator**





In [3]:
def build_generator():
  model = Sequential()

  model.add(Dense(256, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))

  model.summary()
  return model

generator = build_generator()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        25,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 784)            │       201,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 28, 28, 1)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 362,000 (1.38 MB)

 Trainable params: 360,464 (1.38 MB)

 Non-trainable params: 1,536 (6.00 KB)

Setelah itu, saya membangun model generator. Generator berfungsi untuk mengubah vektor acak (noise) menjadi gambar baru yang menyerupai digit asli dari dataset MNIST. Arsitektur generator dibentuk menggunakan model Sequential dengan beberapa lapisan Dense untuk menambah kompleksitas representasi data. Setiap lapisan disertai fungsi aktivasi LeakyReLU agar aliran gradien tetap berjalan meskipun input bernilai negatif, serta lapisan BatchNormalization dengan momentum 0.8 untuk menjaga kestabilan distribusi aktivasi selama proses pelatihan. Pada lapisan terakhir, digunakan fungsi aktivasi tanh karena nilai keluaran generator harus berada dalam rentang -1 hingga 1, sesuai dengan proses normalisasi data MNIST. Setelah itu, hasil keluaran diubah bentuknya menggunakan Reshape(img_shape) agar menjadi gambar berukuran 28x28 piksel dengan satu kanal warna. Generator inilah yang nantinya akan menghasilkan gambar sintetis yang diharapkan menyerupai data asli.

## **4) Building Discriminator**

In [4]:
def build_discriminator():
  model = Sequential()

  model.add(Flatten(input_shape=img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(256))
  model.add(Dense(1, activation='sigmoid'))

  model.summary()
  return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 533,505 (2.04 MB)

 Trainable params: 533,505 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

Kemudian, saya membangun model discriminator yang berfungsi sebagai pengklasifikasi biner untuk membedakan antara gambar asli dan gambar hasil generator. Arsitektur discriminator diawali dengan lapisan Flatten yang mengubah input gambar berukuran 28x28x1 menjadi vektor satu dimensi. Lapisan berikutnya terdiri dari Dense(512) dan Dense(256) untuk memperkuat kemampuan diskriminasi model. Aktivasi LeakyReLU kembali digunakan agar model dapat belajar lebih baik pada berbagai nilai input. Pada lapisan output digunakan Dense(1, activation='sigmoid'), di mana fungsi aktivasi sigmoid menghasilkan probabilitas antara 0 dan 1. Nilai mendekati 1 menunjukkan gambar dianggap asli, sedangkan mendekati 0 menunjukkan gambar dianggap palsu. Model discriminator kemudian dikompilasi menggunakan fungsi loss binary_crossentropy dan optimizer Adam, dengan tambahan metrik akurasi untuk memantau performa pelatihan.

## **5) Connecting Neural Networks to build GAN**

In [5]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer=adam)
GAN.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 28, 28, 1)      │       362,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 1)              │       533,505 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 895,505 (3.42 MB)

 Trainable params: 360,464 (1.38 MB)

 Non-trainable params: 535,041 (2.04 MB)

Setelah kedua model dibentuk, saya menggabungkan keduanya menjadi satu kesatuan model GAN. Dalam struktur ini, generator dan discriminator dihubungkan secara berurutan menggunakan model Sequential. Discriminator kemudian dikunci dengan perintah trainable = False agar bobotnya tidak diperbarui ketika melatih generator. Hal ini penting karena pada tahap pelatihan gabungan, hanya generator yang perlu belajar untuk menipu discriminator. Model gabungan GAN kemudian dikompilasi menggunakan fungsi loss binary_crossentropy dan optimizer Adam. Tujuan utama dari model ini adalah melatih generator agar mampu menghasilkan gambar yang cukup realistis sehingga dapat menipu discriminator dengan baik.

## **6) Outputting Images**


In [6]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001
    print("%.8f" % save_name)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            # axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images/%.8f.png" % save_name)
    print('saved')
    plt.close()

Setelah model siap, saya membuat fungsi tambahan bernama save_imgs() yang berfungsi untuk menampilkan dan menyimpan hasil gambar yang dihasilkan oleh generator. Fungsi ini mengambil sejumlah vektor acak, kemudian menghasilkan gambar-gambar sintetis melalui generator. Nilai piksel hasil keluaran generator yang semula berada pada rentang [-1, 1] diubah menjadi [0, 1] agar dapat ditampilkan dengan benar menggunakan matplotlib. Sebanyak 25 gambar diatur dalam format grid berukuran 5x5 dan disimpan ke dalam folder generated_images dengan nama file unik berbasis nilai variabel save_name. Proses ini memungkinkan saya untuk melihat perkembangan kualitas gambar hasil generator pada interval tertentu selama pelatihan.

## **7) Training GAN**

In [7]:
def train(epochs, batch_size=64, save_interval=200):
    (X_train, _), (_, _) = mnist.load_data()
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)  # wajib

    valid = np.ones((batch_size, 1))
    fakes = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise, verbose=0)

        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = GAN.train_on_batch(noise, valid)

        print(f"{epoch} [D loss: {d_loss[0]:.4f}, acc: {100*d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")

        if epoch % save_interval == 0:
            save_imgs(epoch)


In [10]:
train(30000, batch_size=64, save_interval=200)

/usr/local/lib/python3.12/dist-packages/keras/src/backend/tensorflow/trainer.py:83: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Streaming output truncated to the last 5000 lines.
25072 [D loss: 6.7251, acc: 2.18%] [G loss: 0.0023]
25073 [D loss: 6.7252, acc: 2.18%] [G loss: 0.0023]
25074 [D loss: 6.7253, acc: 2.18%] [G loss: 0.0023]
25075 [D loss: 6.7254, acc: 2.18%] [G loss: 0.0023]
25076 [D loss: 6.7254, acc: 2.18%] [G loss: 0.0023]
25077 [D loss: 6.7255, acc: 2.18%] [G loss: 0.0023]
25078 [D loss: 6.7256, acc: 2.18%] [G loss: 0.0023]
25079 [D loss: 6.7257, acc: 2.18%] [G loss: 0.0023]
25080 [D loss: 6.7258, acc: 2.18%] [G loss: 0.0023]
25081 [D loss: 6.7259, acc: 2.18%] [G loss: 0.0023]
25082 [D loss: 6.7260, acc: 2.18%] [G loss: 0.0023]
25083 [D loss: 6.7261, acc: 2.18%] [G loss: 0.0023]
25084 [D loss: 6.7262, acc: 2.18%] [G loss: 0.0023]
25085 [D loss: 6.7263, acc: 2.18%] [G loss: 0.0023]
25086 [D loss: 6.7263, acc: 2.18%] [G loss: 0.0023]
25087 [D loss: 6.7264, acc: 2.18%] [G loss: 0.0023]
25088 [D loss: 6.7265, acc: 2.18%] [G loss: 0.0023]
25089 [D loss: 6.7266, acc: 2.18%] [G loss: 0.0023]
25090 [D loss

Tahap berikutnya adalah mendefinisikan fungsi train() yang digunakan untuk melatih model GAN. Dalam fungsi ini, saya terlebih dahulu memuat dataset MNIST, kemudian menormalisasikannya ke dalam rentang nilai [-1, 1]. Agar bentuk data sesuai dengan arsitektur model, saya menambahkan satu dimensi menggunakan np.expand_dims() sehingga data gambar menjadi berukuran 28x28x1. Selanjutnya, saya mendefinisikan label valid dengan nilai 1 sebagai representasi gambar asli dan label fakes dengan nilai 0 sebagai representasi gambar palsu. Pada setiap iterasi pelatihan, saya mengambil batch acak dari data asli dan menghasilkan batch gambar palsu menggunakan generator. Discriminator kemudian dilatih dua kali pada setiap epoch, pertama dengan gambar asli (label 1) dan kedua dengan gambar palsu (label 0). Setelah itu, generator dilatih melalui model GAN untuk memaksimalkan peluang bahwa gambar palsu dianggap asli oleh discriminator. Nilai loss dan akurasi dari discriminator serta loss dari generator dicetak setiap epoch untuk memantau proses pelatihan. Fungsi save_imgs() dipanggil setiap beberapa interval untuk menyimpan hasil visualisasi perkembangan generator.

### **8) Making GIF**

In [11]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

/tmp/ipython-input-1295497862.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
/tmp/ipython-input-1295497862.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


Pada bagian terakhir, saya membuat animasi berformat GIF yang menampilkan seluruh gambar hasil generator selama pelatihan. Proses ini dilakukan menggunakan library imageio dan glob. Saya memuat seluruh file gambar .png yang tersimpan dalam folder generated_images, kemudian menyusunnya secara berurutan berdasarkan waktu pembuatan. Setiap gambar ditambahkan ke dalam file animasi bernama dcgan.gif. Hasil akhir dari proses ini adalah animasi yang memperlihatkan evolusi kemampuan generator dari awal hingga akhir pelatihan, di mana gambar yang dihasilkan akan semakin menyerupai digit tulisan tangan asli seiring bertambahnya jumlah epoch pelatihan.